In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
from google.colab import userdata

token = userdata.get('hf_token')

login(token=token)

print("Logged into Hugging Face!")


Logged into Hugging Face!


In [ ]:
!pip3 install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)

def load_tsv(filepath):
    """
    Load a TSV file into a pandas DataFrame.
    Args:
        filepath (str): Path to the TSV file.
    Returns:
        pd.DataFrame: Loaded DataFrame.
    """
    return pd.read_csv(filepath, sep="\t")

train_tsv_path = "/content/drive/MyDrive/orientation-tr-train.tsv"
test_tsv_path = "/content/drive/MyDrive/orientation-tr-test.tsv"

# Load training and test datasets
train_data = load_tsv(train_tsv_path)
test_data = load_tsv(test_tsv_path)


In [ ]:
# Stratified split: 80% train, 20% validation
train_split, validation_split = train_test_split(
    train_data,
    test_size=0.2,
    stratify=train_data["label"],
    random_state=42
)

# Check sizes
print(f"Training Data: {len(train_split)} samples")
print(f"Validation Data: {len(validation_split)} samples")


Training Data: 12910 samples
Validation Data: 3228 samples


In [ ]:
train_split.head()

,id,speaker,sex,text,text_en,label
14033,tr14033,45e4423a87a662cc6af8f82d0303524d,M,Size şunu söyleyeyim: Belki cumhuriyet tarihin...,I'll tell you this: maybe this is the first ti...,1
8460,tr08460,67d62e3c40143111f9ec51c7df8ad4cc,M,"Değerli Başkanım, teşekkür ederim. <p> Açılan ...","Dear President, thank you. <p> I've been promi...",1
6370,tr06370,ad20d5f3022726a758dc2a11bb70ba27,M,"Değerli milletvekilleri, Parlamentolararası Bi...","Thank you, Mr. President. <p> Dear MPs, due to...",1
14678,tr14678,c83bc106e69ebc01bbf04c0a82989597,M,Türkiye Cumhuriyeti tarihi boyunca elbette sağ...,"Mr. President, I'll begin by welcoming our Vet...",1
6154,tr06154,bd7dde993fd5896cff942aea6c796be2,M,"Sayın Başkan, değerli milletvekilleri; bütçesi...","Mr. President, the high-value lawmakers, every...",0


In [ ]:
def preprocess_data(data, tokenizer, is_test=False):
    """
    Preprocess and tokenize the dataset.
    Args:
        data (pd.DataFrame): Input DataFrame with 'text_en' (and optionally 'label') columns.
        tokenizer: Hugging Face tokenizer.
        is_test (bool): If True, processes the test dataset without labels.
    Returns:
        Dataset: Tokenized Hugging Face Dataset.
    """
    def tokenize_function(examples):
        return tokenizer(examples["text_en"], padding="max_length", truncation=True, max_length=512)

    dataset = Dataset.from_pandas(data)

    # Tokenize the data
    dataset = dataset.map(tokenize_function, batched=True)

    # Format for PyTorch
    if not is_test:
        dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    else:
        dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

    return dataset


In [ ]:
# Initialize tokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocess train and validation splits
train_dataset = preprocess_data(train_split, tokenizer)
validation_dataset = preprocess_data(validation_split, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/12910 [00:00<?, ? examples/s]

Map:   0%|          | 0/3228 [00:00<?, ? examples/s]

In [ ]:
# Display the first few samples of the dataset
print(train_dataset[:5])  # Displays the first 5 samples


{'label': tensor([1, 1, 1, 1, 0]), 'input_ids': tensor([[     0,     87,     25,  ...,      1,      1,      1],
        [     0,    262,    147,  ..., 163965,      4,      2],
        [     0,  25689,    398,  ...,    442,    831,      2],
        [     0,   8414,      5,  ...,    642,    765,      2],
        [     0,   8414,      5,  ...,  70796,    136,      2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    """
    Computes evaluation metrics including accuracy.
    Args:
        pred: Predictions from the model.
    Returns:
        dict: A dictionary with accuracy, precision, recall, and F1 score.
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(axis=1)

    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
from transformers import EarlyStoppingCallback

# Load pre-trained multilingual model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Training arguments for XLM-R
training_args = TrainingArguments(
    lr_scheduler_type="cosine",
    warmup_steps=100,
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    warmup_ratio=0.06,
    logging_dir="./logs",
    save_strategy="epoch",
    fp16=True,
    load_best_model_at_end=True,
    report_to="none",
    logging_steps=50,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]

)

# Fine-tune the model
trainer.train()


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-3a05387827c9>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.479700,0.380662,0.825898,0.825420,0.888711,0.855897
2,0.359600,0.339511,0.855638,0.918743,0.824814,0.869248
3,0.292500,0.323992,0.863073,0.884780,0.879127,0.881944
4,0.247600,0.352900,0.865242,0.902398,0.861555,0.881504
5,0.237500,0.348633,0.865242,0.897959,0.866880,0.882146


TrainOutput(global_step=4035, training_loss=0.3501053434322315, metrics={'train_runtime': 2134.8442, 'train_samples_per_second': 30.236, 'train_steps_per_second': 1.89, 'total_flos': 1.6983818623488e+16, 'train_loss': 0.3501053434322315, 'epoch': 5.0})

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.3239923119544983, 'eval_accuracy': 0.8630731102850062, 'eval_precision': 0.8847802786709539, 'eval_recall': 0.8791267305644302, 'eval_f1': 0.8819444444444444, 'eval_runtime': 22.4596, 'eval_samples_per_second': 143.725, 'eval_steps_per_second': 4.497, 'epoch': 5.0}


In [ ]:
from google.colab import drive
import json

# Path to save the file in Google Drive
output_path = "/content/drive/MyDrive/evaluation_results_task1_xlm_tr_en.json"

# Save evaluation results to Google Drive
with open(output_path, "w") as f:
    json.dump(results, f, indent=4)

print(f"Evaluation results saved to '{output_path}'.")


Evaluation results saved to '/content/drive/MyDrive/evaluation_results_task1_xlm_tr_en.json'.


In [ ]:
# Preprocess test data
test_dataset = preprocess_data(test_data, tokenizer, True)

# Perform inference
test_predictions = trainer.predict(test_dataset)
test_data["predictions"] = test_predictions.predictions.argmax(axis=1)

# Path to save the file in Google Drive
output_path = "/content/drive/MyDrive/test_predictions_tr_en_xlm_task1.csv"

# Save test predictions to Google Drive
test_data[["id", "text", "predictions"]].to_csv(output_path, index=False)

print(f"Test predictions saved to '{output_path}'.")



Map:   0%|          | 0/2008 [00:00<?, ? examples/s]

Test predictions saved to '/content/drive/MyDrive/test_predictions_tr_en_xlm_task1.csv'.


In [ ]:
# Perform inference on the test dataset
predictions = trainer.predict(test_dataset)

# Extract predicted labels
predicted_labels = predictions.predictions.argmax(axis=1)

# Count occurrences of 0 and 1
label_counts = pd.Series(predicted_labels).value_counts()

# Display the counts
print(f"Predicted Label Counts:")
print(f"Label 0 (Left): {label_counts.get(0, 0)}")
print(f"Label 1 (Right): {label_counts.get(1, 0)}")


Predicted Label Counts:
Label 0 (Left): 927
Label 1 (Right): 1081
